# Proceso GLUE Eventos Amplitude

## 1. Cargamos las librerias

In [31]:
%%capture
!pip install -q awswrangler

In [32]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import boto3
import ast
from datetime import datetime, timedelta
import awswrangler as wr
from itertools import chain
import gc
import sys
import time
from sklearn import preprocessing

glue = boto3.client('glue')
s3 = boto3.resource('s3')
ssm = boto3.client('ssm') 
lakeformation = boto3.client('lakeformation')

In [34]:
import boto3
s3 = boto3.resource('s3')

# Guardar el archivo whl
s3.meta.client.upload_file('glue_python_libs-0.1-py3-none-any.whl', 
                           'uala-arg-datalake-aiml-recommendations-stage-test', #bucket
                           'artifacts/code/amplitude/glue_python_libs-0.1-py3-none-any.whl')

### 2. Armamos el proceso en "GLUE LTV-RFM AR"

In [88]:
%%writefile get_data_amplitude.py


import sys
import pyspark.sql.functions as func
from awsglue.dynamicframe import DynamicFrame
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.sql.types import *
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from pyspark.sql import SparkSession
from awsglue.job import Job
import pyspark.sql.functions as F
import json
import boto3
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import gc
from pyspark.conf import SparkConf
import pandas as pd
import os
from io import BytesIO
import awswrangler as wr

os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=com.amazonaws:aws-java-sdk-bundle:1.11.271,org.apache.hadoop:hadoop-aws:3.1.2 pyspark-shell"


print('Lectura de parámetros')

# ----------------------------------------------------------------------------------
print('NOW:', datetime.now())

args = getResolvedOptions(sys.argv,
                          ['bucket_amplitude_data', 
                           'today', 
                           'kms_key_arn', 
                           'recommendations_bucket'])

bucket_amplitude_data = args['bucket_amplitude_data']
recommendations_bucket = args['recommendations_bucket']
kms_key_id = args['kms_key_arn']
today = args['today']


#--------------------------------------------------------------------------------------------------------------


#https://stackoverflow.com/questions/52932459/accessing-s3-bucket-from-local-pyspark-using-assume-role


print('Crear objetos S3-ssm')
# ----------------------------------------------------------------------------------
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
ssm = boto3.client('ssm')

#--------------------------------------------------------------------------------------------------------------
print('Parámetros:')
path_key_amplitude = 'ar/amplitude/tb_ar_amplitude_events_stage/'

## FECHAS INTERVALO
#print('1. CALCULO DE FECHAS')
##Today llevado al primero del mes menos 1 día
#today = datetime.strptime(today, '%Y-%m-%d').date().replace(day=1)
#last_day=(today-pd.offsets.DateOffset(days=1)).date()
##
#first_day=(last_day-pd.offsets.DateOffset(days=365)).date()
#
#print('2. Intevalo de fechas analizada: ',first_day,'y',last_day)

def first_and_last(today):
    fecha=datetime.strptime(today, '%Y-%m-%d').date()
    first_day=fecha.replace(day=1)
    next_month = fecha.replace(day=28) + timedelta(days=4)
    last_day_of_month = next_month - timedelta(days=next_month.day)
    return first_day,last_day_of_month

print('Declaración de funciones')
def list_objects_function(buckets_, first_day, last_day, keys_, retrieve_last=False):
    
    sts = boto3.client('sts')
    response = sts.assume_role(
        RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
        RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
        DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
    )

    s3 = boto3.client(
        's3',
        aws_access_key_id=response['Credentials']['AccessKeyId'],
        aws_secret_access_key=response['Credentials']['SecretAccessKey'],
        aws_session_token=response['Credentials']['SessionToken']
    )


    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_amplitude_data, Prefix=path_key_amplitude)
    files_in_bucket=[]
    for page in pages:
        files_page=[key['Key'] for key in page['Contents']]
        files_in_bucket+=files_page
        files_objets = [f"s3a://{bucket_amplitude_data}/" + i for i in files_in_bucket if
                            (keys_ in i)  and (i.find('.parquet') >= 0)]
        df_bucket_files = pd.DataFrame({
                'key': [i[:(i.find('dt=') + 14)] for i in files_objets],
                'path': files_objets,
                'date': pd.to_datetime([i[(i.find('dt=') + 3):(i.find('dt=') + 13)] for i in files_objets])
            })
        files=list(df_bucket_files.loc[df_bucket_files['date'].between(str(first_day),str(last_day)),'path'].values)
    return files



map_events = {
    "cuoti_selecciona_elegircuotas" : "cuotificaciones",
    "cuoti_sigue_seleccion_consumos" : "cuotificaciones",
    "prestamos_selecciona_simular_prestamo": "prestamos",
    "prestamos_espera": "prestamos",
    "general_ingresa_promociones": "promociones",
    "recargas_click_empezar": "recargas",
    "recargas_click_repetir": "recargas",
    "transferencia_selecciona_tieneuala": "transferencia_c2c",
    "transferencia_selecciona_notieneuala": "transferencia_cvu",
    "general_ingresa_cobros": "cobros",
    "cobros_acepta_tyc" : "cobros",
    "cobros_elige_link": "cobros",
    "cobros_elige_mpos": "cobros",
    "pagos_empezar": "pago_servicios",
    "click_inversiones":"inversiones"
}

eventos_recommendations = list(map_events.keys())
#-----------------------------------------------------------------------------------------------------------------
first_day,last_day = first_and_last(today)
print('Primer dia',first_day)
print('Ultimo dia',last_day)

files_objets_amplitude = list_objects_function(bucket_amplitude_data, first_day, last_day ,path_key_amplitude)

print(f'Hay {len(files_objets_amplitude)} archivos de survival en la carpeta')


#df_amplitude = spark.read.parquet(*files_objets_amplitude).select(['user_id',"os_name","event_type","event_time"])

sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-spark', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

print('Spark Configuración')

spark_conf = SparkConf().setAll([
  ("spark.hadoop.fs.s3.enableServerSideEncryption", "true"),
  ("spark.hadoop.fs.s3.serverSideEncryption.kms.keyId", kms_key_id)
])

  
sc = SparkContext(conf=spark_conf) 
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()

spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", response["Credentials"]["AccessKeyId"])
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", response["Credentials"]["SecretAccessKey"])
spark._jsc.hadoopConfiguration().set("fs.s3a.session.token",  response["Credentials"]["SessionToken"])

print(f"Hadoop version = {sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

df_amplitude = spark.read.parquet(*files_objets_amplitude).select(['user_id',"os_name","event_type","event_time"])


df_amplitude=df_amplitude.filter(df_amplitude.event_type.isin(eventos_recommendations))

df_amplitude = df_amplitude.withColumn('year_month', F.date_format(df_amplitude.event_time,'YYYY-MM'))

df_amplitude = df_amplitude.drop("event_time")

df_amplitude = df_amplitude.na.replace(map_events,1,"event_type")

df_amplitude = (df_amplitude    
      .groupBy(['user_id', 'event_type', 'year_month'])
      .agg(F.count('event_type').alias('cant'),
           F.max('os_name').alias('os_name'))
      .groupBy(['user_id','year_month','os_name'])
      .pivot("event_type")
      .agg(F.sum('cant'))
      .na.fill(0)
      )
print(spark.sparkContext.getConf().getAll())


#### NUEVA INSTANCIA boto3 para usar buckets en stage #####
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
ssm = boto3.client('ssm')
df_pandas=df_amplitude.toPandas()
df_pandas['dt'] = first_day
wr.s3.to_parquet(df_pandas,
                        path='s3://{}/data/raw/amplitude/'.format(recommendations_bucket),
                        dataset=True,
                        partition_cols=['dt'],
                        mode="append",
                        concurrent_partitioning=True,
                        index=False)

print('Ubicación files', f's3://{recommendations_bucket}/data/raw/amplitude/dt={str(first_day)}')

#DELETE $FOLDER$
def retrieve_files(path, file_type, list_dates):
    bucket=path.split('/')[2]
    prefix='/'.join(path.split('/')[3:])
    list_objects=list(s3.Bucket(bucket).objects.all())
    list_objects=[f's3://{bucket}/{i.key}' for i in list_objects if ((i.key.find(prefix)>=0) & any(x in i.key.lower() for x in list_dates) & (i.key.find(file_type)>=0))]
    return list_objects


delete_files = retrieve_files(path=f's3://{recommendations_bucket}/data/', file_type='$folder$', list_dates=['$folder$'])
print('Files to delete', delete_files)
files_keys=[]
for i in range(0,len(delete_files)):
    files_keys=files_keys+[{'Key':('/').join(delete_files[i].split('/')[3:])}]
if len(files_keys)>0:
    s3_client.delete_objects(Bucket=recommendations_bucket,
                             Delete={'Objects':files_keys})
del delete_files
gc.collect()

print(df_amplitude.show())
print((df_amplitude.count(), len(df_amplitude.columns)))

Overwriting get_data_amplitude.py


In [89]:
job_name='test-job_recommendations_amplitude'

In [90]:
 #borrar job
glue.delete_job(
    JobName=job_name
)

{'JobName': 'test-job_recommendations_amplitude',
 'ResponseMetadata': {'RequestId': '8e100a3d-1dec-4962-91fd-42bbcbad1a79',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 13 Jul 2021 16:07:23 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '48',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8e100a3d-1dec-4962-91fd-42bbcbad1a79'},
  'RetryAttempts': 0}}

## 3. Generamos los parametros

In [91]:
today = '2021-01-10'
bucket_amplitude_data='uala-arg-datalake-stage-prod'  ## AFIP, GP, etc
recommendations_bucket='uala-arg-datalake-aiml-recommendations-stage-test'  # Para outputs
kms_key_arn='arn:aws:kms:us-east-1:322149183112:key/9cc44b23-c5e9-46cb-9987-0982d21f8d00' ## key para desencriptar

In [92]:
s3 = boto3.resource('s3')

# Guardar el archivo .py
s3.meta.client.upload_file('get_data_amplitude.py', 
                           recommendations_bucket, #bucket
                           'artifacts/code/amplitude/get_data_amplitude.py' #key+filename
)
print('.py uploaded')

.py uploaded


## 4. Creamos el job de GLUE

In [93]:
job = glue.create_job(Name=job_name, 
                      GlueVersion='2.0',
                      Role='iam_r_uala_arg_datalake_stage_glue',
                      Command={'Name': 'glueetl',
                               'ScriptLocation': f's3://{recommendations_bucket}/artifacts/code/amplitude/get_data_amplitude.py'},
                      DefaultArguments={
                        '--additional-python-modules': 'pip,setuptools,pyarrow==2,awswrangler==2.8.0,numpy==1.19.1'},
                      MaxCapacity=1
                      )

In [84]:
job_run = glue.start_job_run(
    JobName = job_name,
    Arguments = {
        '--today':today,
        '--bucket_amplitude_data': bucket_amplitude_data,
        '--recommendations_bucket': recommendations_bucket,
        '--kms_key_arn': kms_key_arn
    } 
)

In [85]:
print(job_run)

{'JobRunId': 'jr_51f6c6e542a4610dfb2b18a6a72e3a156374dcbdf76a55fb1dfc940edd8fc7c0', 'ResponseMetadata': {'RequestId': '8b567d77-10f6-44d3-a12f-9b12fe01aa58', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 13 Jul 2021 16:04:57 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': '8b567d77-10f6-44d3-a12f-9b12fe01aa58'}, 'RetryAttempts': 0}}


In [86]:
MAX_WAIT_TIME=time.time() + 60*10 # 1 hour
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
    status = response['JobRun']['JobRunState']
    print('Job run: {}'.format(status))
    
    if status == 'SUCCEEDED' or status == 'FAILED':
        break
        
    time.sleep(45)

Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: FAILED


## 5. Controlamos la carga de datos en el bucket


In [ ]:
df=wr.s3.read_parquet(f's3://test-uala-arg-datalake-aiml-recommendations/data/raw/amplitude',dataset=True,partition_filter=lambda x: '2021-01-01' in x["dt"])

In [62]:
df.head(5)

,user_id,year_month,os_name,cobros,cuotificaciones,inversiones,pago_servicios,prestamos,promociones,recargas,transferencia_c2c,transferencia_cvu,dt
0,00360d0c-2638-458b-a1b6-144cb3ba4b78,2021-01,ios,0,0,4,0,0,0,0,4,1,2021-01-01
1,00509604-ce55-47db-93f6-6bca3867ef05,2021-01,android,0,0,0,0,0,0,0,10,4,2021-01-01
2,005e8359-5408-4b28-b2a0-a5ef6955f158,2021-01,android,0,0,0,0,0,0,0,2,1,2021-01-01
3,0072cfda-2213-4b1a-b4bf-3969ccacde3f,2021-01,android,2,0,0,0,0,0,4,4,1,2021-01-01
4,0074aa89-9148-4d94-9bee-e55bd591d038,2021-01,android,0,2,13,2,0,0,5,10,4,2021-01-01


## 6. Carga histórica


In [94]:
bucket_amplitude_data='uala-arg-datalake-stage-prod'  ## AFIP, GP, etc
recommendations_bucket='uala-arg-datalake-aiml-recommendations-stage-test'  # Para outputs
kms_key_arn='arn:aws:kms:us-east-1:322149183112:key/9cc44b23-c5e9-46cb-9987-0982d21f8d00' ## key para desencriptar
list_fechas=['2020-06-01','2020-07-01','2020-08-01','2020-09-01','2020-10-01','2020-11-01','2020-12-01',
            '2021-01-01','2021-02-01','2021-03-01','2021-04-01','2021-05-01']
job_name='test-job_recommendations_amplitude'

In [25]:
for value in list_fechas:
    print("Procesando:",value[:7])
    job_run = glue.start_job_run(
        JobName = job_name,
        Arguments = {
            '--today':value,
            '--bucket_amplitude_data': bucket_amplitude_data,
            '--recommendations_bucket': recommendations_bucket,
            '--kms_key_arn': kms_key_arn
        } 
    )
    MAX_WAIT_TIME= 3600 # 1 hour
    max_time = time.time() + MAX_WAIT_TIME
    while time.time() < max_time:
        response=glue.get_job_run(JobName=job_name, RunId=job_run['JobRunId'])
        status = response['JobRun']['JobRunState']
        print('Job run: {}'.format(status))

        if status == 'SUCCEEDED':
            time.sleep(60)
            break
        elif status == 'FAILED':
            print ("Error para fecha:",value[:7]," \n" )
            sys.exit(1)
        time.sleep(45)

Procesando: 2020-05
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-06
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-07
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-08
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-09
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-10
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-11
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesando: 2020-12
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: RUNNING
Job run: SUCCEEDED
Procesan

In [26]:
#pre
112085647+91896875+107893286+74785390+79832664+80947544+60897141+72981539+86242174+84235181+86334578

938132019

In [27]:
#post
1028539+942427+935155+1134141+862378+860491+429003+640624+828413+828413+882853

9372437

In [ ]:
pre limpieza 60897141  colummas 31
post limpieza 429003 colummas 7
pre limpieza 72981539 columnas 31
post limpieza 640624 columnas 9
pre limpieza 86242174 columnas 31
post limpieza 828413 columnas 10
pre limpieza 84235181 columnas 31
post limpieza 828413 columnas 10
pre limpieza 86334578 columnas 31
post limpieza 882853 columnas 10

In [55]:
sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

s3 = boto3.client(
    's3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
)


# Get the path to the file
s3_response_object = s3.get_object(Bucket='uala-arg-datalake-stage-prod', Key='ar/afip/tb_ar_afip_actividades/dt=2021-03-16/')

# Read your file, i.e. convert it from a stream to bytes using .read()
df = s3_response_object['Body'].read()

# Read your file using BytesIO
df = pd.read_parquet(BytesIO(df))


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [ ]:
import io
import pandas as pd


sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

buffer = io.BytesIO()
s3 = boto3.resource( 's3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
)

objeto = s3.Object('ACA EL BUCKET', 'aca/va/el/path.parquet')
objeto.download_fileobj(buffer)
df = pd.read_parquet(buffer)

In [98]:
s3=boto3.resource('s3')
df=pd.read_parquet('s3://uala-arg-datalake-aiml-recommendations-stage-test/data/raw/amplitude/dt=2020-06-01/aecc4cdd5d1b4401987f1fbdd45d266c.snappy.parquet')

PermissionError: Forbidden

In [310]:
today = '2021-01-10'
def first_and_last(today):
    fecha=datetime.strptime(today, '%Y-%m-%d').date()
    first_day=fecha.replace(day=1)
    next_month = fecha.replace(day=28) + timedelta(days=4)
    last_day_of_month = next_month - timedelta(days=next_month.day)
    return first_day,last_day_of_month
first_day,last_day = first_and_last(today)

keys_='ar/amplitude/tb_ar_amplitude_events_stage/'

sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

s3 = boto3.client(
    's3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
)


paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_amplitude_data, Prefix='ar/amplitude/tb_ar_amplitude_events_stage/')
files_in_bucket=[]
for page in pages:
    files_page=[key['Key'] for key in page['Contents']]
    files_in_bucket+=files_page
files_objets = [f"s3://{bucket_amplitude_data}/" + i for i in files_in_bucket if
                    (keys_ in i)  and (i.find('.parquet') >= 0)]
df_bucket_files = pd.DataFrame({
        'key': [i[:(i.find('dt=') + 14)] for i in files_objets],
        'path': files_objets,
        'date': pd.to_datetime([i[(i.find('dt=') + 3):(i.find('dt=') + 13)] for i in files_objets])
    })
#files=list(df_bucket_files.loc[df_bucket_files['date'].between(str(first_day),str(last_day)),'path'].values)
files=list(df_bucket_files.loc[:,'path'].values)
files

['s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/1e4fdbb9488148c68c9d29622a49edc1.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/375adef7d335448889605a3f7f7614b6.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/8952a93f527449aa8bcc72c015f72c2b.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/ac377b2e101346e6be51fef2bb0e4bb3.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/be8c30ddb4244c9c9854dd230a8d655a.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/eadbf4f44b6b47648c3d3a570709518a.snappy.parquet',
 's3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/f7bf94c36e7c4201a592d38e11037035.snappy.parquet',
 's3:/

In [18]:
sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)

s3 = boto3.client(
    's3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
)
files_bucket=[key['Key'] for key in s3.list_objects(Bucket=bucket_amplitude_data)['Contents']]

In [22]:
keys_

'ar/amplitude/tb_ar_amplitude_events_stage/'

In [17]:
bucket_amplitude_data

'uala-arg-datalake-stage-prod'

In [218]:
sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)
def get_matching_s3_keys(bucket, prefix, suffix):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    s3 = boto3.client('s3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
                     )
    kwargs = {'Bucket': bucket}

    # If the prefix is a single string (not a tuple of strings), we can
    # do the filtering directly in the S3 API.
    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:

        # The S3 API response is a large blob of metadata.
        # 'Contents' contains information about the listed objects.
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix) and key.endswith(suffix):
                yield key

        # The S3 API is paginated, returning up to 1000 keys at a time.
        # Pass the continuation token into the next response, until we
        # reach the final page (when this field is missing).
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [223]:

for key in get_matching_s3_keys(bucket='bukkit', prefix='images/', suffix='.parquet'):
    print('hola')

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [221]:
asd

<generator object get_matching_s3_keys at 0x7f5cbb6900f8>

In [214]:
asd

<generator object keys at 0x7f5cb95baba0>

In [ ]:
files_in_bucket=[key['Key'] for key in s3.list_objects(Bucket=bucket_amplitude_data)['Contents']]
files_in_bucket

In [188]:
keys_

'ar/amplitude/tb_ar_amplitude_events_stage/'

In [9]:

sts = boto3.client('sts')
response = sts.assume_role(
    RoleArn='arn:aws:iam::514405401387:role/aws-rol-ml-read-stage-prod', #es el rol que existe en produccion por el cual "nos hacemos pasar" para acceder a los buckets de s3
    RoleSessionName='sesion-dsr-recomendaciones', # nombre que le damos a la sesión
    DurationSeconds=3600 # es el tiempo que dura la sesion por default si no especificamos este parámetro.
)
s3 = boto3.client(
    's3',
    aws_access_key_id=response['Credentials']['AccessKeyId'],
    aws_secret_access_key=response['Credentials']['SecretAccessKey'],
    aws_session_token=response['Credentials']['SessionToken']
)
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_amplitude_data)
files_in_bucket=[]
for page in pages:
    files_page=[key['Key'] for key in s3.list_objects(Bucket=bucket_amplitude_data)['Contents']]
    files_in_bucket+=files_page

ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied

In [180]:
keys_

'ar/amplitude/tb_ar_amplitude_events_stage/'

In [168]:
keys_

'ar/amplitude/tb_ar_amplitude_events_stage/'

In [47]:
df_bucket_files.date.value_counts().sort_index()


2019-11-20     7
2019-11-21    13
2019-11-22    18
2019-11-23    14
2019-11-24    12
              ..
2021-07-04    24
2021-07-05    28
2021-07-06    24
2021-07-07    24
2021-07-08    14
Name: date, Length: 597, dtype: int64

In [45]:

df_bucket_files[(df_bucket_files.date>='2021-01-01')&(df_bucket_files.date<='2021-01-31')].date.value_counts().sort_index()
#df_bucket_files.date.value_counts().sort_index()

2021-01-01    24
2021-01-02    24
2021-01-03    24
2021-01-04    24
2021-01-05    24
2021-01-06    24
2021-01-07    24
2021-01-08    24
2021-01-09    24
2021-01-10    24
2021-01-11    24
2021-01-12    24
2021-01-13    24
2021-01-14    24
2021-01-15    24
2021-01-16    24
2021-01-17    24
2021-01-18    24
2021-01-19    24
2021-01-20    24
2021-01-21    24
2021-01-22    24
2021-01-23    24
2021-01-24    24
2021-01-25    24
2021-01-26    24
2021-01-27     8
2021-01-28    24
2021-01-29    24
2021-01-30    24
2021-01-31    24
Name: date, dtype: int64

In [37]:
df_bucket_files.dtypes

key             object
path            object
date    datetime64[ns]
dtype: object

In [11]:
paginator = s3.get_paginator('list_objects')
pages = paginator.paginate(Bucket=bucket_amplitude_data, Prefix=keys_)
files_in_bucket=[]
for page in pages:
    files_page=[key['Key'] for key in s3.list_objects(Bucket=bucket_amplitude_data)['Contents']]
    files_in_bucket+=files_page

ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

In [55]:
pd.set_option('display.max_columns', 500)  # or 1000
pd.set_option('display.max_rows', 500)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199
df_bucket_files.path

0        s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/1e4fdbb9488148c68c9d29622a49edc1.snappy.parquet
1        s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/375adef7d335448889605a3f7f7614b6.snappy.parquet
2        s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/8952a93f527449aa8bcc72c015f72c2b.snappy.parquet
3        s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/ac377b2e101346e6be51fef2bb0e4bb3.snappy.parquet
4        s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2019-11-20/be8c30ddb4244c9c9854dd230a8d655a.snappy.parquet
                                                                           ...                                                                    
14234    s3://uala-arg-datalake-stage-prod/ar/amplitude/tb_ar_amplitude_events_stage/dt=2021-07-08/ae0624f3786f440e820